In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy
using PDFIO
using Unzip
using Interpolations
using Plots
using TensorCast
#plotlyjs()
data_dir = "C:\\Cross-Section-Data\\EXFOR\\"

"C:\\Cross-Section-Data\\EXFOR\\"

Is it faster use a matrix or run more loops? Probably just to run more loops. 

In [17]:
file_path = data_dir * "neutrons\\092_U_238.c4"
file_as_vector = readlines(file_path)
max_line_length = maximum(length.(file_as_vector))
l = length(file_as_vector)
spacing_specifiers = [index for index in 1:length(file_as_vector) 
if check_line(file_as_vector[index], "# Prj")]
index = spacing_specifiers[1]
spacing_dict = make_spacing_dict(file_as_vector[index], file_as_vector[index + 1])
spacing_keys = [key for key in keys(spacing_dict)]
lines_of_data = tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
data_dict = Dict([])
filler = [data_dict[key] = [read_datum(file_as_vector[line_num][spacing_dict[key]])
            for line_num in index + 2: index + 1 + lines_of_data] for key in spacing_keys]   
data_dict

Dict{Any, Any} with 18 entries:
  "Targ"       => [92238.0, 92238.0, 92238.0, 92238.0, 92238.0, 92238.0, 92238.…
  "Entry"      => SubString{String}["V0015", "V0015", "V0015", "V0015", "V0015"…
  "dCos/LO"    => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "I78"        => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "dEnergy"    => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "dELV/HL"    => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "M"          => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "Refer (YY)" => SubString{String}["F.Manero,ET.AL. (72)", "F.Manero,ET.AL. (7…
  "Prj"        => [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.…
  "Energy"     => Any[1.25e6, 1.275e6, 1.3e6, 1.325e6, 1.35e6, 1.375e6, 1.4e6, …
  "Cos/LO"     => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "ELV/HL"     => SubString{String}["", "", "", "", "", "", "", "", "", ""  …

#Design

    Looping check_line() through each line in the file detects where each dataset begins. 

    Reading the line above each output of check_line() gives the number of rows in each dataset.

    make_spacing_dict() gets the names of the columns and the instructions for how to read each dataset.

    read_dataset() reads each and constructs a dataframe from each dataset.
    
    combine_datasets() creates vertically concatenates each dataset.

    read_exfor_file() runs all of the above in order to return a single DataFrame of a given file path.
     

In [93]:
function check_line(line, start)
    if length(line) > length(start) - 1
        return (line[1:length(start)] == start)
    end
    return false
end

function make_spacing_dict(line1, line2)
    spacing_ends = [collect(out)[1] for out in findall(">", line2)]
    spacing_starts = append!([1], [collect(out)[1] for out in findall("<", line2)])
    #Find where some of the spacings begin and end based on the arrows
    if length(spacing_ends) != length(spacing_starts)
        print("Error: improper formatting")
        return 
    end
    indices_with_gaps = [index for index in  1:length(spacing_ends)-1 if 
                    spacing_ends[index] != spacing_starts[index+1]-1]
    #Some of the spacings are instead denoted by the letter o instead of arrows
    missing_spacings = [spacing_ends[index]+1:spacing_starts[index+1]-1 for index in indices_with_gaps]
    spacings = append!(missing_spacings, 
    [spacing_starts[i]:spacing_ends[i] for i in 1:length(spacing_ends)])
    spacing_dict = Dict([])
    #Make a dictionary where the keys are name for each column and the values are the indices of the columns
    spacing_names = [spacing_dict[strip(line1[spacing], [' ', '#'])] = spacing for spacing in spacings]
    return spacing_dict
end

function read_datum(datum)
    #Reads a single datum from a line of data
    datum = strip(datum, [' '])
    out = tryparse(Float64, datum)
    if out == nothing
        return datum 
    end
    return out
end

function read_dataset(spacing_specifier, file_as_vector)
    index = spacing_specifier
    spacing_dict = make_spacing_dict(file_as_vector[index], file_as_vector[index + 1])
    lines_of_data = tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
    data_dict = Dict([])
    filler = [data_dict[key] = [read_datum(file_as_vector[line_num][spacing_dict[key]])
            for line_num in index + 2: index + 1 + lines_of_data] for key in keys(spacing_dict)]    
    return data_dict
end

read_dataset (generic function with 1 method)

It it faster to use 1 or loop for $n$ list comprehensions? 

In [149]:
function read_exfor_file(file_path)
    #Reads an EXFOR file and returns a dictionary of data
    file_as_vector = readlines(file_path)
    max_line_length = maximum(length.(file_as_vector))
    spacing_specifiers = [index for index in 1:length(file_as_vector) 
    if check_line(file_as_vector[index], "# Prj")]
    #Split the file into lines
    #Find the line that starts with "Data"
    #==
    for index in 1:length(file_as_vector)
        if check_line(file_as_vector[index], "# Prj")
            rows_in_dataset = tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
            spacing_dict = make_spacing_dict(file_as_vector[index], file_as_vector[index + 1])
            data_dict = Dict([])
            filler = [data_dict[key] =  [read_datum(
                    file_as_matrix[index + 2 : index + 1 + rows_in_dataset,spacing_dict[key]])
                    for key in keys(spacing_dict)]]    
        end
    end
    ==#
    if spacing_specifiers == []
        print("Error: no data found")
        return
    end
    rows_in_each_dataset = [tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
                            for index in spacing_specifiers]    
    top_of_each_dataset = [sum(rows_in_each_dataset[1:index]) 
                            for index in 1:length(rows_in_each_dataset)]
    #Each dataset should have the same column names
    spacing_dict = make_spacing_dict(file_as_vector[spacing_specifiers[1]], 
                    file_as_vector[spacing_specifiers[1] + 1])
    spacing_keys = [key for key in keys(spacing_dict)]
    #make an empty dataframe to fill up with the data
    rows_in_df = sum(rows_in_each_dataset)
    df = DataFrame([Vector{Union{Missing, Float64, String, SubString{String}}}(missing, rows_in_df) 
                    for _ in 1:length(spacing_keys)], spacing_keys)
    #Fill the dataframe with the data
    top_dataset = read_dataset(spacing_specifiers[1], file_as_vector)
    _ = [df[1:rows_in_each_dataset[1], key] = top_dataset[key] for key in spacing_keys]
    print("hi")
    _ = [[df[top_of_each_dataset[index-1]:top_of_each_dataset[index]-1, key] = 
            read_dataset(spacing_specifiers[index], file_as_vector)[key] 
            for key in spacing_keys] for index in 2:length(top_of_each_dataset)]
    return df
end

read_exfor_file (generic function with 1 method)

In [150]:
read_exfor_file(file_path)

hi

,Targ,Entry,dCos/LO,I78,dEnergy,dELV/HL,M,Refer (YY)
,Union…?,Union…?,Union…?,Union…?,Union…?,Union…?,Union…?,Union…?
1,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
2,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
3,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
4,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
5,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
6,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
7,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
8,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
9,1001.0,K2066,,,4.0000+7,,,"S.Homma,ET.AL. (74)"
